# K-Nearest-Neighbors Membership Inference Experiment

In [1]:
import pandas as pd
import sklearn.ensemble as es
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
import logging
from sklearn.neighbors import KNeighborsClassifier

In [2]:
threads = 15

In [3]:
%run experiment_setup.ipynb

INFO:xai-privacy:Loading dataset 1: heart disease (numeric features) ...
INFO:xai-privacy:Loading dataset 2: census income (categorical features) ...


Feature Age: removed 0 rows for missing values.
Feature RestingBP: removed 59 rows for missing values.
Feature Cholesterol: removed 27 rows for missing values.
Feature FastingBS: add unknown category 2.0
Feature RestingECG: add unknown category 3.0
Feature MaxHR: removed 0 rows for missing values.
Feature Oldpeak: removed 7 rows for missing values.
Feature ST_Slope: add unknown category 4.0
Feature CA: add unknown category 4.0
Feature Thal: add unknown category 8.0
Dropped 271 of 1097
Dropped 273 of 1097
Dropped 277 of 1097
Dropped: 2399 of 32561
census: Dropped 3848 of 30162
num: Dropped 19859 of 30162
cat: Dropped 12136 of 30162


In [4]:
logger = logging.getLogger('xai-privacy')

This notebook will go through the experiment for membership inference with KNN as an explanation. Membership inference means an attacker with access to the explanation can tell for a sample whether it was included in the training data or not.

The idea for KNN membership inference is as follows: Enter the given sample and check whether the sample is returned as one of its own nearest neighbors. If it is, it is part of the training data. Otherwise it is not.

First, we have to create our own wrapper class for the default scikit-learn KNN classifier. This is necessary because the scikit-learn implementation only returns the indices of the nearest neighbors. However, in order to be useful explanation to a user, the actual feature values of the nearest neighbors need to be returned. This is done by the wrapper class.

In [5]:
# define own KNN explainer that provides the k nearest neighbors to a query point
class KnnExplainer():
    def __init__(self, data, outcome_name):
        features = data.drop(outcome_name, axis=1)
        labels = data[outcome_name]
        self._knn_model = KNeighborsClassifier().fit(features, labels)
        self._data = data
        
    def explain(self, sample_df):
        nei_indices = self._knn_model.kneighbors(X=sample_df, return_distance=False)
        neighbors = self._data.iloc[nei_indices[0], :]
        return neighbors

Then, we implement the `train_explainer` and `membership_inference_attack_no_model_access` functions:

In [6]:
class KnnMembershipInference(MembershipInference):
    def train_explainer(self, data_train, model):
        return KnnExplainer(data_train, self.outcome_name)
        
    @staticmethod
    def membership_inference_attack_no_model_access(explainer, samples_df):
        inferred_membership = np.empty(len(samples_df))
        
        for index in range(len(samples_df)):
            # needs double brackets so that iloc returns a dataframe instead of a series
            sample_df = samples_df.iloc[[index], :]

            logger.debug(f'Checking sample {index}: {sample_df.to_numpy()[0]}')

            # explainer does not need target for explanation (remove last column)
            neighbors = explainer.explain(sample_df.drop(sample_df.columns[-1], axis=1))
            
            logger.debug(f'K nearest neighbors: \n {neighbors.to_numpy()}')
            
            # check if the sample itself is part of it's own nearest neighbors. In that case, it is part of the training data.
            # otherwise it isn't.
            result = np.isclose(neighbors.to_numpy().astype(float), sample_df.to_numpy().astype(float)).all(axis=1).any()

            logger.debug('Inferred membership: %s' % result)
            inferred_membership[index] = result
        
        return inferred_membership

# Dataset 1: Heart Disease

We now generate a KNN explaination for a random sample from the training data as a demonstration of the concept.

In [7]:
features = data_heart.drop(outcome_name_heart, axis=1)
labels = data_heart[outcome_name_heart]

# Train a random forest on training data.
model = es.RandomForestClassifier(random_state=0)
model = model.fit(features, labels)

# Train explainer
exp = KnnExplainer(data_heart, outcome_name_heart)

given_sample = features.sample()
print(f'Given sample: {given_sample.to_numpy()}')

pred = model.predict(given_sample)
print(f'Model prediction: {pred[0]}')

neighbors = exp.explain(given_sample)
print(f'Nearest Neighbors: \n {neighbors.to_numpy()}')

Given sample: [[ 68.   1.   3. 150. 195.   1.   0. 132.   0.   0.   4.   4.   6.]]
Model prediction: 1.0
Nearest Neighbors: 
 [[ 68.    1.    3.  150.  195.    1.    0.  132.    0.    0.    4.    4.
    6.    1. ]
 [ 64.    1.    4.  150.  193.    0.    1.  135.    1.    0.5   2.    4.
    8.    1. ]
 [ 68.    1.    4.  144.  193.    1.    0.  141.    0.    3.4   2.    2.
    7.    1. ]
 [ 54.    1.    2.  160.  195.    0.    1.  130.    0.    1.    1.    4.
    8.    0. ]
 [ 55.    1.    4.  140.  201.    0.    0.  130.    1.    3.    2.    4.
    8.    1. ]]


We will now do a small proof of concept of the experiment with logging enabled to demonstrate how it works.

In [8]:
logger.setLevel(logging.DEBUG)

EXP = KnnMembershipInference(data_heart, numeric_features_heart, outcome_name_heart, random_state=0)
EXP.membership_inference_experiment(num_queries=10, model=DecisionTreeClassifier(random_state=0), model_access=False, threads=1)

logger.setLevel(logging.INFO)

DEBUG:xai-privacy:[[ 57.   1.   4. 140. 214.   0.   1. 144.   1.   2.   2.   4.   6.   1.]] taken from training data
DEBUG:xai-privacy:[[ 63.   1.   4. 160. 230.   1.   0. 105.   1.   1.   2.   4.   8.   1.]] taken from test data
DEBUG:xai-privacy:[[ 67.   1.   1. 145.   0.   0.   2. 125.   0.   0.   2.   4.   3.   1.]] taken from training data
DEBUG:xai-privacy:[[ 67.   1.   4. 160. 384.   1.   1. 130.   1.   0.   2.   4.   8.   1.]] taken from test data
DEBUG:xai-privacy:[[ 66.    1.    4.  120.  302.    0.    2.  151.    0.    0.4   2.    0.
    3.    0. ]] taken from training data
DEBUG:xai-privacy:[[ 53.   1.   4.  80.   0.   2.   0. 141.   1.   2.   3.   4.   8.   0.]] taken from test data
DEBUG:xai-privacy:[[ 50.   1.   4. 144. 349.   0.   2. 120.   1.   1.   1.   4.   7.   1.]] taken from training data
DEBUG:xai-privacy:[[ 41.   0.   3. 112. 268.   0.   2. 172.   1.   0.   1.   0.   3.   0.]] taken from test data
DEBUG:xai-privacy:[[ 55.   0.   2. 110. 344.   0.   1. 160.   0. 

Total time: 0.21s (training model: 0.02s, training explainer: 0.00s, experiment: 0.18s)
Accuracy: 1.0, precision: 1.0, recall: 1.0


The proof of concept should show that the membership inference function predicts membership very accurately.

Now we begin executing the actual experiment. We begin by defining the table that will hold the results for all our different experiment variations. Then we execute all variations of the experiment for this dataset. We vary the model between a decision tree, a random forest and a neural network. Each model uses the default configuration of scikit-learn.

In [9]:
results_ = {'dataset': [], 'model': [], 'accuracy': [], 'precision': [], 'recall': []}

results = pd.DataFrame(data = results_)

In [10]:
dataset_dicts = [data_heart_dict, data_heart_num_dict, data_heart_cat_dict, data_census_dict, data_census_num_dict, data_census_cat_dict]

dt_dict = {'name': 'decision tree', 'model': DecisionTreeClassifier}
rf_dict = {'name': 'random forest', 'model': es.RandomForestClassifier}
nn_dict = {'name': 'neural network', 'model': MLPClassifier}

model_dicts = [dt_dict, rf_dict, nn_dict]

In [11]:
# This will run the experiment for each dataset and model combination

results = run_all_experiments(KnnMembershipInference, dataset_dicts, model_dicts, random_state=0, num_queries=None, model_access=False, threads=threads, results_table=results)

dataset: heart, model: decision tree
Total time: 0.88s (training model: 0.02s, training explainer: 0.00s, experiment: 0.85s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: random forest
Total time: 0.88s (training model: 0.16s, training explainer: 0.00s, experiment: 0.72s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart, model: neural network


/opt/conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 1.34s (training model: 0.69s, training explainer: 0.00s, experiment: 0.65s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart numeric, model: decision tree
Total time: 0.47s (training model: 0.01s, training explainer: 0.00s, experiment: 0.46s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart numeric, model: random forest
Total time: 0.62s (training model: 0.17s, training explainer: 0.00s, experiment: 0.46s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart numeric, model: neural network
Total time: 1.09s (training model: 0.41s, training explainer: 0.00s, experiment: 0.67s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart categorical, model: decision tree
Total time: 0.52s (training model: 0.01s, training explainer: 0.00s, experiment: 0.50s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: heart categorical, model: random forest
Total time: 0.90s (training model: 0.19s, training explainer: 0.00s, experiment: 0.70s)
Accuracy: 1.0, precisio

/opt/conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 1.61s (training model: 0.99s, training explainer: 0.00s, experiment: 0.62s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census, model: decision tree
Total time: 10.31s (training model: 0.47s, training explainer: 0.02s, experiment: 9.82s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census, model: random forest
Total time: 16.19s (training model: 6.43s, training explainer: 0.02s, experiment: 9.74s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census, model: neural network


...

Total time: 34.83s (training model: 25.13s, training explainer: 0.03s, experiment: 9.67s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census numeric, model: decision tree
Total time: 3.61s (training model: 0.01s, training explainer: 0.00s, experiment: 3.59s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census numeric, model: random forest
Total time: 4.09s (training model: 0.46s, training explainer: 0.00s, experiment: 3.62s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census numeric, model: neural network


/opt/conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 10.36s (training model: 6.87s, training explainer: 0.01s, experiment: 3.49s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census categorical, model: decision tree
Total time: 7.04s (training model: 0.21s, training explainer: 0.01s, experiment: 6.82s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census categorical, model: random forest
Total time: 9.93s (training model: 3.29s, training explainer: 0.01s, experiment: 6.63s)
Accuracy: 1.0, precision: 1.0, recall: 1.0
dataset: census categorical, model: neural network


/opt/conda/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Total time: 24.76s (training model: 17.98s, training explainer: 0.02s, experiment: 6.76s)
Accuracy: 1.0, precision: 1.0, recall: 1.0


# Results

The results of all variations of the membership inference experiment with KNN. In every experiment, we executed the membership inference attack on each sample of the training data and each sample of the test data. Both datasets are of equal size and originate from the same source dataset.

Accuracy is the percentage of samples whose membership (true or false) was correctly inferred. An algorithm guessing at random would achieve an accuracy of 50 percent.

Precision is the percentage of predicted training samples that is actually in the training data.

Recall is the percentage of training samples whose membership (true) was correctly inferred.

In [12]:
results

,dataset,model,accuracy,precision,recall
0,heart,decision tree,1.0,1.0,1.0
1,heart,random forest,1.0,1.0,1.0
2,heart,neural network,1.0,1.0,1.0
3,heart numeric,decision tree,1.0,1.0,1.0
4,heart numeric,random forest,1.0,1.0,1.0
5,heart numeric,neural network,1.0,1.0,1.0
6,heart categorical,decision tree,1.0,1.0,1.0
7,heart categorical,random forest,1.0,1.0,1.0
8,heart categorical,neural network,1.0,1.0,1.0
9,census,decision tree,1.0,1.0,1.0


In [13]:
results.to_csv('results/1-4-knn-membership-inference-results.csv', index=False, na_rep='NaN', float_format='%.3f')

# Discussion

Just as expected, the attack has an accuracy of 100%. No false positives or false negatives occur.